# Question 1
## Use PCA

Sve olthis question on Kaggle! Submit your predictions in Kaggle.
Download the notebook and submit it separately along with the other questions.

https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview

Use everything that you have learnt so far.

Analyse the data. Drop, scale or normalize features. Merge features if they correlated to make some meaningful features.<br> 
**Use PCA to reduce the dimensionality of the problem.**

Try out all the regression models. Use cross validation to find best parameters for each model.

In [45]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder,StandardScaler

In [46]:
train = pd.read_csv(r"C:\Users\91900\Downloads\train (1).csv")
test = pd.read_csv(r"C:\Users\91900\Downloads\test (1).csv")

In [47]:
nan_percentage = train.isna().mean().round(4) * 100

# Defining a threshold; columns with more than 50% NaN values will be dropped
threshold = 50
columns_to_drop = nan_percentage[nan_percentage > threshold].index

# Dropping the columns with too many NaN values
train_reduced = train.drop(columns=columns_to_drop)

# Filling remaining NaN values with the median for numeric columns and the mode for categorical columns
numeric_columns = train_reduced.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = train_reduced.select_dtypes(include=['object']).columns

train_reduced[numeric_columns] = train_reduced[numeric_columns].fillna(train_reduced[numeric_columns].median())
train_reduced[categorical_columns] = train_reduced[categorical_columns].fillna(train_reduced[categorical_columns].mode().iloc[0])


In [48]:
train_reduced.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [49]:
nan_percentage = test.isna().mean().round(4) * 100

# Defining a threshold; columns with more than 50% NaN values will be dropped
threshold = 50
columns_to_drop = nan_percentage[nan_percentage > threshold].index

# Dropping the columns with too many NaN values
test_reduced = test.drop(columns=columns_to_drop)

# Filling remaining NaN values with the median for numeric columns and the mode for categorical columns
numeric_columns = test_reduced.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = test_reduced.select_dtypes(include=['object']).columns

test_reduced[numeric_columns] = test_reduced[numeric_columns].fillna(test_reduced[numeric_columns].median())
test_reduced[categorical_columns] = test_reduced[categorical_columns].fillna(test_reduced[categorical_columns].mode().iloc[0])


In [50]:
test_reduced.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,120,0,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,...,36,0,0,0,0,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,...,34,0,0,0,0,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,...,36,0,0,0,0,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,...,82,0,0,144,0,0,1,2010,WD,Normal


In [51]:
X = train_reduced.drop(['SalePrice','Id'],axis = 1)
y = train_reduced['SalePrice']

In [52]:
categoricalCols = X.select_dtypes(include=object).columns
numericalCols = X.select_dtypes(include=['int64', 'float64']).columns

cat = X[categoricalCols]
num = X[numericalCols]

In [53]:
le = LabelEncoder()

for i in cat.columns:
    cat.loc[:, i] = le.fit_transform(cat[i])

In [54]:
cat.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,3,1,3,3,0,4,0,5,2,2,...,2,6,2,1,1,4,4,2,8,4
1,3,1,3,3,0,2,0,24,1,2,...,3,6,4,1,1,4,4,2,8,4
2,3,1,0,3,0,4,0,5,2,2,...,2,6,4,1,1,4,4,2,8,4
3,3,1,0,3,0,0,0,6,2,2,...,2,6,2,5,2,4,4,2,8,0
4,3,1,0,3,0,2,0,15,2,2,...,2,6,4,1,1,4,4,2,8,4


In [55]:
scaler = StandardScaler()

scaled_num = scaler.fit_transform(num)
scaled_num = pd.DataFrame(scaled_num, columns = num.columns)

In [56]:
scaled_num.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,0.073375,-0.220875,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.514104,0.575425,-0.288653,...,0.351000,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,0.138777
1,-0.872563,0.460320,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.570750,1.171992,-0.288653,...,-0.060731,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-0.489110,-0.614439
2,0.073375,-0.084636,0.073480,0.651479,-0.517200,0.984752,0.830215,0.325915,0.092907,-0.288653,...,0.631726,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.990891,0.138777
3,0.309859,-0.447940,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.570750,-0.499274,-0.288653,...,0.790804,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,-1.367655
4,0.073375,0.641972,0.375148,1.374795,-0.517200,0.951632,0.733308,1.366489,0.463568,-0.288653,...,1.698485,0.780197,0.563760,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,2.100892,0.138777


In [57]:
if not scaled_num.index.equals(cat.index):
    scaled_num = scaled_num.reset_index(drop=True)
    cat = cat.reset_index(drop=True)

modified_X = pd.concat([scaled_num, cat], axis=1)
modified_X.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,0.073375,-0.220875,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.514104,0.575425,-0.288653,...,2,6,2,1,1,4,4,2,8,4
1,-0.872563,0.460320,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.570750,1.171992,-0.288653,...,3,6,4,1,1,4,4,2,8,4
2,0.073375,-0.084636,0.073480,0.651479,-0.517200,0.984752,0.830215,0.325915,0.092907,-0.288653,...,2,6,4,1,1,4,4,2,8,4
3,0.309859,-0.447940,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.570750,-0.499274,-0.288653,...,2,6,2,5,2,4,4,2,8,0
4,0.073375,0.641972,0.375148,1.374795,-0.517200,0.951632,0.733308,1.366489,0.463568,-0.288653,...,2,6,4,1,1,4,4,2,8,4


In [58]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(modified_X,y, test_size=0.2, random_state=5)

In [60]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [66]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred= lr.predict(X_test)
print(accuracy_score(y_test,y_pred))

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [63]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 10)
new_X_train = pca.fit_transform(X_train)
new_X_test = pca.transform(X_test)

logr2= logreg()
logr2.fit(new_X_train, y_train)
y_pred2= logr2.predict(new_X_test)
print(accuracy_score(y_test,y_pred2))

0.010273972602739725


c:\Users\91900\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
